In [1]:
from utils.data_preprocess import *
from utils.tf_helpers import create_model_checkpoint, evaluate_preds
import pandas as pd
import numpy as np

In [2]:
df = DataPreprocess('ETHBTC-5m-data.csv').get_ohlcv()

In [3]:
X, y = make_windows(df, 12)

In [4]:
X.head()

,Open,High,Low,Volume,Close12,Close11,Close10,Close9,Close8,Close7,Close6,Close5,Close4,Close3,Close2,Close1
Timestamp,,,,,,,,,,,,,,,,
2017-07-14 05:00:00,0.085874,0.086205,0.085874,1.304,0.086347,0.086380,0.086350,0.086314,0.086180,0.086000,0.085811,0.086000,0.085128,0.086400,0.080001,0.080000
2017-07-14 05:05:00,0.085816,0.085816,0.084824,9.770,0.085978,0.086347,0.086380,0.086350,0.086314,0.086180,0.086000,0.085811,0.086000,0.085128,0.086400,0.080001
2017-07-14 05:10:00,0.084803,0.084803,0.084608,5.446,0.084824,0.085978,0.086347,0.086380,0.086350,0.086314,0.086180,0.086000,0.085811,0.086000,0.085128,0.086400
2017-07-14 05:15:00,0.084581,0.086196,0.084581,8.833,0.084680,0.084824,0.085978,0.086347,0.086380,0.086350,0.086314,0.086180,0.086000,0.085811,0.086000,0.085128
2017-07-14 05:20:00,0.085590,0.085810,0.085496,4.461,0.086196,0.084680,0.084824,0.085978,0.086347,0.086380,0.086350,0.086314,0.086180,0.086000,0.085811,0.086000


In [5]:
y.head()

Timestamp
2017-07-14 05:00:00    0.085978
2017-07-14 05:05:00    0.084824
2017-07-14 05:10:00    0.084680
2017-07-14 05:15:00    0.086196
2017-07-14 05:20:00    0.085496
Name: Close, dtype: float32

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

In [9]:
def simple_baseline():
    input = tf.keras.Input(shape=(16,), name='input_layer')
    x = layers.Dense(256, activation='relu')(input)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(input, output, name='simple_baseline_model')
    
    return model

In [10]:
model = simple_baseline()
model.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

In [ ]:
model.fit()